# Desafío 2 - Properati

## Modelo base: Precios hedónicos

Con el objeto de interpretar las relaciones entre la variable objetivo y las descriptivas, partimos de un modelo relativamente sencillo, conocido como Modelo de Precios Hedónicos y lo ejecutamos con statsmodels para verificar la significancia de las variables. Dicho modelo tiene la forma:
$$ \ln{(\text{price_usd_per_m2})} = \beta_0+\beta_1\times\text{rooms}+\beta_2\times\text{surface_total_in_m2}+\beta_{3i}\times\text{property_type}_i+\beta_{4i}\times\text{localidad}_i$$

donde $\text{property_type}$ y $\text{localidad}$ son variables categóricas.

Para ello, importamos el dataset limpio y las librerías necesarias

In [15]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('df2.csv')
df.sample(20)

E:\Miniconda3\envs\ds37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,property_type,place_name,geonames_id,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,...,Estrenar,Gimnasio,Lavadero,Parrilla,Pileta,SUM,Seguridad,log_price,log_price_aprox_usd,log_price_usd_per_m2
46381,54851,house,San Vicente,3428056.0,650000.00,USD,11468925.00,650000.00,3400.0,400.0,...,0,0,0,0,0,0,0,13.384728,13.384728,5.253197
83830,101254,house,San Isidro,3428983.0,365000.00,USD,6440242.50,365000.00,217.0,217.0,...,0,0,0,1,1,0,1,12.807653,12.807653,NaN
64303,76254,apartment,Lomas de Zamora,NaN,140000.00,USD,2470230.00,140000.00,53.0,53.0,...,0,0,0,0,0,0,0,11.849398,11.849398,7.879106
15553,18081,apartment,Pinamar,3429971.0,255000.00,USD,4499347.50,255000.00,130.0,130.0,...,0,0,0,1,1,0,1,12.449019,12.449019,7.581484
22161,26400,apartment,Boedo,3436003.0,425687.00,USD,7511034.27,425687.00,30.0,30.0,...,0,0,0,0,0,0,0,12.961460,12.961460,9.560262
57622,67816,house,Castelar,3435607.0,115000.00,USD,2029117.50,115000.00,80.0,80.0,...,0,0,1,1,0,0,0,11.652687,11.652687,7.270661
70513,84112,apartment,Córdoba,3860255.0,60009.53,USD,1058838.15,60009.53,58.0,47.0,...,0,0,0,0,0,0,0,11.002259,11.002259,6.941816
21294,24755,apartment,Villa Carlos Paz,3832791.0,220000.00,USD,3881790.00,220000.00,140.0,140.0,...,0,0,0,0,1,0,1,12.301383,12.301383,7.359740
51419,60566,apartment,Recoleta,3429595.0,180000.00,USD,3176010.00,180000.00,40.0,40.0,...,0,0,1,0,0,0,1,12.100712,12.100712,8.411833
58847,69332,apartment,Monserrat,3430570.0,178000.00,USD,3140721.00,178000.00,139.0,139.0,...,0,0,0,0,0,0,0,12.089539,12.089539,NaN


In [8]:
for i in df.columns:
    print(i, df[i].dtype)

Unnamed: 0 int64
property_type object
place_name object
geonames_id float64
price float64
currency object
price_aprox_local_currency float64
price_aprox_usd float64
surface_total_in_m2 float64
surface_covered_in_m2 float64
price_usd_per_m2 float64
price_per_m2 float64
rooms float64
description object
title object
lat float64
lon float64
precio_regex float64
moneda object
provincia object
localidad object
barrio object
Amenities int64
Cochera int64
Estrenar int64
Gimnasio int64
Lavadero int64
Parrilla int64
Pileta int64
SUM int64
Seguridad int64
log_price float64
log_price_aprox_usd float64
log_price_usd_per_m2 float64


##### Nos aseguramos que la variable `price_usd_per_m2` esté completa y obtenemos las variables logarítmicas actualizadas

In [16]:
df['price_usd_per_m2'] = df.price/df.surface_total_in_m2
df['log_price'] = df.price.apply(np.log)
df['log_price_aprox_usd'] = df.price_aprox_usd.apply(np.log)
df['log_price_usd_per_m2'] = df.price_usd_per_m2.apply(np.log)

##### Generamos el modelo descrito arriba con statsmodels

In [17]:
model_base = smf.ols('log_price_usd_per_m2 ~ rooms + surface_total_in_m2 + C(property_type) + C(localidad)', data=df)
model_base.fit().summary2()

E:\Miniconda3\envs\ds37\lib\site-packages\statsmodels\base\model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
E:\Miniconda3\envs\ds37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
E:\Miniconda3\envs\ds37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
E:\Miniconda3\envs\ds37\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                  Results: Ordinary least squares
====================================================================================================
Model:                         OLS                           Adj. R-squared:              0.608     
Dependent Variable:            log_price_usd_per_m2          AIC:                         37834.9565
Date:                          2019-10-07 14:47              BIC:                         40132.1360
No. Observations:              27823                         Log-Likelihood:              -18638.   
Df Model:                      278                           F-statistic:                 156.2     
Df Residuals:                  27544                         Prob (F-statistic):          0.00      
R-squared:                     0.612                         Scale:                       0.22582   
----------------------------------------------------------------------------------------------------
                                                     Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
----------------------------------------------------------------------------------------------------
Intercept                                            7.2676   0.0710 102.3506 0.0000  7.1284  7.4068
C(property_type)[T.apartment]                        0.4421   0.0115  38.5368 0.0000  0.4196  0.4646
C(property_type)[T.house]                           -0.4133   0.0130 -31.8025 0.0000 -0.4388 -0.3878
C(property_type)[T.store]                            0.2470   0.0583   4.2337 0.0000  0.1327  0.3614
C(localidad)[T.Achiras]                             -0.0000   0.0000  -3.3149 0.0009 -0.0000 -0.0000
C(localidad)[T.Adolfo Alsina]                        0.0000   0.0000   3.9535 0.0001  0.0000  0.0000
C(localidad)[T.Agronomía]                            0.0514   0.1252   0.4106 0.6814 -0.1939  0.2967
C(localidad)[T.Agua Blanca]                          0.0000   0.0000   5.6179 0.0000  0.0000  0.0000
C(localidad)[T.Agua de Oro]                         -0.0000   0.0000  -1.8984 0.0576 -0.0000  0.0000
C(localidad)[T.Aguas Verdes]                        -1.3817   0.2063  -6.6961 0.0000 -1.7861 -0.9772
C(localidad)[T.Aldea Brasilera]                     -1.1788   0.4804  -2.4538 0.0141 -2.1205 -0.2372
C(localidad)[T.Aldea Salto]                         -0.0000   0.0000  -1.5924 0.1113 -0.0000  0.0000
C(localidad)[T.Alejandro Roca]                       3.2505   0.4805   6.7651 0.0000  2.3088  4.1923
C(localidad)[T.Almafuerte]                          -0.0000   0.0000  -2.7024 0.0069 -0.0000 -0.0000
C(localidad)[T.Almagro]                              0.0331   0.0730   0.4531 0.6505 -0.1101  0.1762
C(localidad)[T.Almirante Brown]                     -0.3498   0.0793  -4.4093 0.0000 -0.5052 -0.1943
C(localidad)[T.Alta Gracia]                         -1.8059   0.4804  -3.7591 0.0002 -2.7476 -0.8643
C(localidad)[T.Aluminé]                             -0.0000   0.0000  -2.0404 0.0413 -0.0000 -0.0000
C(localidad)[T.Alvear]                              -0.2281   0.4805  -0.4747 0.6350 -1.1699  0.7137
C(localidad)[T.Anisacate]                            0.0000   0.0000   1.2575 0.2086 -0.0000  0.0000
C(localidad)[T.Apóstoles]                           -0.2609   0.4805  -0.5430 0.5871 -1.2026  0.6808
C(localidad)[T.Arequito]                            -0.3873   0.4804  -0.8064 0.4200 -1.3289  0.5542
C(localidad)[T.Arocena]                              0.0000   0.0000   0.1371 0.8909 -0.0000  0.0000
C(localidad)[T.Arroyo Ceibal]                        0.0000   0.0000   2.2714 0.0231  0.0000  0.0000
C(localidad)[T.Arroyo Leyes]                        -0.0000   0.0000  -2.2694 0.0233 -0.0000 -0.0000
C(localidad)[T.Arroyo Seco]                         -1.2099   0.4805  -2.5182 0.0118 -2.1516 -0.2682
C(localidad)[T.Ascochinga]                          -0.0000   0.0000  -2.0277 0.0426 -0.0000 -0.0000
C(localidad)[T.Ataliva]                        

Como podemos observar en el output anterior, el modelo se está fiteando sobre $27823$ observaciones y estamos describiendo un $0.609 (R^2=0.612)$ de la variación en el precio de los inmuebles.

A continuación probamos ingresar las dummies de Amenities, Seguridad, Cochera, A Estrenar, Gimnasio, Lavadero, Parrilla, Pileta, Sum y una transformación sobre rooms para aumentar el poder descriptivo del modelo. También optamos por no calcular un intercepto, ya que no tiene mucho sentido.

In [24]:
df['rooms2'] = df.rooms**2
df['rooms3'] = df.rooms**3
df['rooms4'] = df.rooms**4
model_base_mod =  smf.ols('log_price_usd_per_m2 ~ rooms + rooms2 + rooms3 + rooms4 + surface_total_in_m2 + \
                            surface_covered_in_m2 + C(property_type) + C(localidad) + Seguridad + \
                            Amenities + Cochera + Estrenar + Gimnasio + Lavadero + Parrilla + Pileta + SUM -1', data=df)
model_base_mod.fit().summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                                  Results: Ordinary least squares
====================================================================================================
Model:                         OLS                           Adj. R-squared:              0.627     
Dependent Variable:            log_price_usd_per_m2          AIC:                         36428.1117
Date:                          2019-10-07 15:31              BIC:                         38832.3282
No. Observations:              27823                         Log-Likelihood:              -17922.   
Df Model:                      291                           F-statistic:                 162.1     
Df Residuals:                  27531                         Prob (F-statistic):          0.00      
R-squared:                     0.631                         Scale:                       0.21459   
----------------------------------------------------------------------------------------------------
                                                     Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
----------------------------------------------------------------------------------------------------
C(property_type)[PH]                                 7.3620   0.0719 102.3437 0.0000  7.2210  7.5030
C(property_type)[apartment]                          7.7575   0.0713 108.7501 0.0000  7.6177  7.8973
C(property_type)[house]                              6.9568   0.0719  96.8225 0.0000  6.8159  7.0976
C(property_type)[store]                              7.6121   0.0900  84.5434 0.0000  7.4356  7.7885
C(localidad)[T.Achiras]                              0.0000   0.0000   1.1678 0.2429 -0.0000  0.0000
C(localidad)[T.Adolfo Alsina]                       -0.0000   0.0000  -7.0432 0.0000 -0.0000 -0.0000
C(localidad)[T.Agronomía]                           -0.0324   0.1221  -0.2656 0.7905 -0.2718  0.2069
C(localidad)[T.Agua Blanca]                          0.0000   0.0000  11.3869 0.0000  0.0000  0.0000
C(localidad)[T.Agua de Oro]                         -0.0000   0.0000  -7.3928 0.0000 -0.0000 -0.0000
C(localidad)[T.Aguas Verdes]                        -1.4193   0.2013  -7.0518 0.0000 -1.8138 -1.0248
C(localidad)[T.Aldea Brasilera]                     -1.2957   0.4684  -2.7662 0.0057 -2.2138 -0.3776
C(localidad)[T.Aldea Salto]                          0.0000   0.0000   8.6526 0.0000  0.0000  0.0000
C(localidad)[T.Alejandro Roca]                       3.2122   0.4684   6.8578 0.0000  2.2941  4.1303
C(localidad)[T.Almafuerte]                          -0.0000   0.0000  -5.5863 0.0000 -0.0000 -0.0000
C(localidad)[T.Almagro]                              0.0512   0.0713   0.7186 0.4724 -0.0885  0.1909
C(localidad)[T.Almirante Brown]                     -0.3531   0.0774  -4.5625 0.0000 -0.5048 -0.2014
C(localidad)[T.Alta Gracia]                         -1.7786   0.4684  -3.7976 0.0001 -2.6966 -0.8606
C(localidad)[T.Aluminé]                              0.0000   0.0000   1.2834 0.1994 -0.0000  0.0000
C(localidad)[T.Alvear]                              -0.3784   0.4686  -0.8074 0.4194 -1.2969  0.5401
C(localidad)[T.Anisacate]                           -0.0000   0.0000  -3.9449 0.0001 -0.0000 -0.0000
C(localidad)[T.Apóstoles]                           -0.2989   0.4685  -0.6380 0.5235 -1.2172  0.6194
C(localidad)[T.Arequito]                            -0.4298   0.4684  -0.9176 0.3588 -1.3479  0.4883
C(localidad)[T.Arocena]                             -0.0000   0.0000  -4.2130 0.0000 -0.0000 -0.0000
C(localidad)[T.Arroyo Ceibal]                       -0.0000   0.0000  -0.8323 0.4052 -0.0000  0.0000
C(localidad)[T.Arroyo Leyes]                         0.0000   0.0000   3.5607 0.0004  0.0000  0.0000
C(localidad)[T.Arroyo Seco]                         -1.3735   0.4685  -2.9318 0.0034 -2.2917 -0.4553
C(localidad)[T.Ascochinga]                          -0.0000   0.0000  -1.6530 0.0983 -0.0000  0.0000
C(localidad)[T.Ataliva]                        

Revisamos por qué hay tantas localidades y barrios que no son significativos. Para ello agrupamos las observaciones por localidad, obtenemos los estadísitcos básicos, filtramos por las que no son NaN y las ordenamos por desvío estándar.

In [39]:
varianzas = df.groupby('localidad').log_price_usd_per_m2.describe()
display(varianzas.shape)
varianzas.loc[varianzas['std'].isnull()==False].sort_values('std', ascending=False)

(519, 8)

,count,mean,std,min,25%,50%,75%,max
localidad,,,,,,,,
La Cumbre,2.0,5.478362,2.792535e+00,3.503741,4.491052,5.478362,6.465672,7.452982
Nono,3.0,5.096565,2.634353e+00,2.525729,3.749775,4.973821,6.381983,7.790144
Brandsen,8.0,6.165967,2.110951e+00,1.487623,5.556291,7.272223,7.482739,7.536364
Plottier,7.0,6.382504,2.015403e+00,2.607617,5.844774,6.860058,7.180629,9.159047
Los Hornillos,2.0,5.425119,1.907923e+00,4.076014,4.750566,5.425119,6.099671,6.774224
...,...,...,...,...,...,...,...,...
Villa Icho Cruz,2.0,5.401889,2.529902e-03,5.400100,5.400995,5.401889,5.402783,5.403678
Dorrego,3.0,5.636283,1.087792e-15,5.636283,5.636283,5.636283,5.636283,5.636283
Timbúes,3.0,4.448693,0.000000e+00,4.448693,4.448693,4.448693,4.448693,4.448693


De las 519 localidades que tenemos, sólo tenemos datos del logaritmo de los precios en 270, lo cual explica el resultado. También podemos observar cuáles son las localidades con mayor desviación estándar.

In [46]:
model_caba_mod =  smf.ols('log_price_usd_per_m2 ~ C(rooms) + surface_total_in_m2 + \
                            surface_covered_in_m2 + C(property_type) + C(localidad) + Seguridad + \
                            Amenities + Cochera + Estrenar + Gimnasio + Lavadero + Parrilla + Pileta + SUM -1', data=df.loc[df.provincia=='Capital Federal'])
model_caba_mod.fit().summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Results: Ordinary least squares
=====================================================================================
Model:                   OLS                       Adj. R-squared:          0.542    
Dependent Variable:      log_price_usd_per_m2      AIC:                     4824.7299
Date:                    2019-10-07 15:58          BIC:                     5451.6631
No. Observations:        11798                     Log-Likelihood:          -2327.4  
Df Model:                84                        F-statistic:             167.4    
Df Residuals:            11713                     Prob (F-statistic):      0.00     
R-squared:               0.546                     Scale:                   0.087500 
-------------------------------------------------------------------------------------
                                      Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
-------------------------------------------------------------------------------------
C(rooms)[1.0]                         7.4157   0.0451 164.4886 0.0000  7.3273  7.5041
C(rooms)[2.0]                         7.3615   0.0450 163.6095 0.0000  7.2733  7.4497
C(rooms)[3.0]                         7.3462   0.0450 163.3752 0.0000  7.2581  7.4344
C(rooms)[4.0]                         7.3068   0.0454 160.9153 0.0000  7.2178  7.3958
C(rooms)[5.0]                         7.2793   0.0469 155.2463 0.0000  7.1874  7.3713
C(rooms)[6.0]                         7.2593   0.0503 144.3765 0.0000  7.1608  7.3579
C(rooms)[7.0]                         7.1618   0.0546 131.2458 0.0000  7.0548  7.2687
C(rooms)[8.0]                         7.1646   0.0689 103.9297 0.0000  7.0295  7.2997
C(rooms)[9.0]                         7.1611   0.1158  61.8440 0.0000  6.9341  7.3881
C(rooms)[10.0]                        7.1051   0.1065  66.7054 0.0000  6.8963  7.3139
C(rooms)[11.0]                        6.5507   0.1423  46.0210 0.0000  6.2717  6.8297
C(rooms)[12.0]                        7.0594   0.1161  60.8285 0.0000  6.8319  7.2869
C(rooms)[13.0]                        6.5683   0.3096  21.2121 0.0000  5.9613  7.1752
C(rooms)[14.0]                        6.4475   0.2155  29.9142 0.0000  6.0250  6.8700
C(rooms)[15.0]                        7.0583   0.2123  33.2538 0.0000  6.6423  7.4744
C(rooms)[17.0]                       -0.0000   0.0000  -4.4324 0.0000 -0.0000 -0.0000
C(rooms)[18.0]                        0.0000   0.0000  23.5161 0.0000  0.0000  0.0000
C(rooms)[22.0]                       -0.0000   0.0000 -27.4541 0.0000 -0.0000 -0.0000
C(property_type)[T.apartment]         0.2254   0.0111  20.3040 0.0000  0.2037  0.2472
C(property_type)[T.house]            -0.1033   0.0181  -5.6961 0.0000 -0.1389 -0.0678
C(property_type)[T.store]             0.1098   0.0450   2.4384 0.0148  0.0215  0.1980
C(localidad)[T.Agronomía]            -0.0053   0.0782  -0.0677 0.9460 -0.1586  0.1480
C(localidad)[T.Almagro]               0.0772   0.0458   1.6878 0.0915 -0.0125  0.1669
C(localidad)[T.Balvanera]            -0.0460   0.0469  -0.9815 0.3264 -0.1378  0.0459
C(localidad)[T.Barracas]             -0.0727   0.0479  -1.5169 0.1293 -0.1665  0.0212
C(localidad)[T.Barrio Norte]          0.3340   0.0457   7.3151 0.0000  0.2445  0.4234
C(localidad)[T.Belgrano]              0.3512   0.0450   7.8036 0.0000  0.2630  0.4394
C(localidad)[T.Boca]                 -0.2559   0.0533  -4.7991 0.0000 -0.3604 -0.1514
C(localidad)[T.Boedo]                 0.3022   0.0505   5.9853 0.0000  0.2032  0.4011
C(localidad)[T.Caballito]             0.1303   0.0450   2.8931 0.0038  0.0420  0.2185
C(localidad)[T.Catalinas]            -0.0619   0.2994  -0.2068 0.8362 -0.6487  0.5249
C(localidad)[T.Centro / Microcentro]  0.0376   0.0509   0.7384 0.4603 -0.0622  0.1375
C(localidad)[T.Chacarita]             0.0635   0.0539   1.1776 0.2390 -0.0422  0.1691
C(localidad)[T.Coghlan]               0.2216   0.0526   4.2098 0.0000  0.1184  0.3248
C(localidad)[T.Colegia

In [44]:
df.loc[df.provincia=='Capital Federal'].groupby('localidad').log_price_usd_per_m2.describe().sort_values('std', ascending=False)

,count,mean,std,min,25%,50%,75%,max
localidad,,,,,,,,
Boedo,215.0,7.755845,0.747558,6.781004,7.313497,7.600902,7.824046,10.449714
Centro / Microcentro,186.0,7.665014,0.630684,3.687629,7.507378,7.702685,7.870453,10.349775
Parque Chacabuco,88.0,7.321837,0.536032,4.511750,7.164348,7.432278,7.666287,8.142500
Parque Avellaneda,30.0,7.119464,0.484047,5.352385,6.967139,7.264430,7.402318,7.786306
Mataderos,259.0,7.178445,0.476265,6.214608,6.804128,7.186927,7.513891,9.721166
Villa Soldati,6.0,6.669731,0.461537,5.999833,6.602692,6.659138,6.664525,7.450080
Once,117.0,7.539732,0.447027,6.186328,7.293418,7.513891,7.706263,9.998798
Boca,167.0,7.274257,0.445558,5.368740,7.138448,7.430557,7.526247,7.924129
Pompeya,31.0,6.900431,0.427604,6.127597,6.696305,6.882858,7.156078,7.708533


##### Generamos dummies por localidad (barrio), tipo de inmueble y cantidad de ambientes para incorporar en scikit learn
Construimos dos dataframes nuevos: 
- prices (que posee todas las variables relacionadas a precio)
- X (que posee todas las varialbes descriptivas a partir de las cuales obtener las dummies)

In [68]:
#df.drop(columns='Unnamed: 0', inplace=True)
prices = df.filter(['price', 'price_aprox_local_currency', 'price_aprox_usd','price_usd_per_m2','price_per_m2', \
               'log_price', 'log_price_aprox_usd', 'log_price_usd_per_m2', 'log_price_usd_per_m2', 'log_price_usd_per_m2 '])
#pd.get_dummies(df, columns=['property_type', 'localidad'], drop_first=True)
X = df.filter(['property_type', 'surface_total_in_m2', 'surface_covered_in_m2', 'rooms','localidad',\
               'barrio', 'Amenities', 'Cochera', 'Estrenar', 'Gimnasio','Lavadero', 'Parrilla', 'Pileta', 'SUM', \
               'Seguridad', 'rooms2', 'rooms3', 'rooms4' ])
X = pd.get_dummies(X.property_type, drop_first=1).join(X).drop('property_type', axis=1)
X = pd.get_dummies(X.localidad, drop_first=1).join(X).drop('localidad', axis=1)

In [69]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


def findAlphaLasso(X,y,randomState=53,tries=5,alphaFrom=0.00001,alphaTo=100000,steps=1000):
    kf = KFold(n_splits=5, shuffle=True, random_state=randomState)
    step_value=(alphaTo-alphaFrom)/steps
    print("Lasso:")
    prevAlphaTest=0
    for i in range(0,tries):
        al_lasso = np.linspace(alphaFrom, alphaTo, steps)
        lm_lasso_cv= LassoCV(alphas=al_lasso, cv=kf, normalize=True)
        lm_lasso_cv.fit(X, y)
        if( round(prevAlphaTest*10000) == round(lm_lasso_cv.alpha_*10000) ):
            return lm_lasso_cv 
        prevAlphaTest=lm_lasso_cv.alpha_
        alphaFrom = prevAlphaTest - step_value*2
        alphaTo = prevAlphaTest   + step_value*2
        step_value=(alphaTo-alphaFrom)/steps
        print("intento {} value {} from {} to {} ".format(i+1,lm_lasso_cv.alpha_,alphaFrom,alphaTo))
    return lm_lasso_cv

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, prices.log_price_usd_per_m2, shuffle=True)
lasso = findAlphaLasso(X_train, y_train)
print(" Score Train Lasso: %.2f\n" % lasso.score(X_train, y_train))
print(" Score Test Lasso: %.2f\n" % lasso.score(X_test, y_test))

Lasso:


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').